In [3]:
import pandas as pd
import requests
from pandas import json_normalize

headers = {
    "User-Agent": "Mozilla/5.0"
}

symbols = ["TSLA", "MSFT", "AAPL", "NVDA"]


# symbols = ['1299.HK', '1788.HK', '9988.HK', '2828.HK', '2800.HK', '3033.HK', '1810.HK', '3690.HK', \
#  '0700.HK', '0981.HK', '9868.H', '0939.HK', '9926.HK', '2318.HK', '1211.HK', '1024.HK', '9992.HK', \
#  '0241.HK', '0388.HK', '3750.HK']

# 3 years
start_unix_date = "1657237004" # unix time 2022.07
end_unix_date = "1751931404" # unix time 2025.07
df_all = pd.DataFrame()

def call_yh_api(symbol, start_date, end_date):
  url = "https://query1.finance.yahoo.com/v8/finance/chart/" + symbol \
    + "?period1=" + start_date + "&period2=" + end_date \
    + "&interval=1d&events=history"
  print(url)
  response = requests.get(url, headers=headers)
  data = response.json()
  return data

def convert_data_to_series(data):
  result = data["chart"]["result"][0]
  timestamps = pd.Series(result["timestamp"])
  highs = pd.Series(result["indicators"]["quote"][0]["high"])
  lows = pd.Series(result["indicators"]["quote"][0]["low"])
  opens = pd.Series(result["indicators"]["quote"][0]["open"])
  closes = pd.Series(result["indicators"]["quote"][0]["close"])
  volumes = pd.Series(result["indicators"]["quote"][0]["volume"])
  return timestamps, highs, lows, opens, closes, volumes

def validate_series(timestamps, highs, lows, opens, closes, volumes):
  if (len(timestamps) != len(highs) or len(highs) != len(lows) \
      or len(lows) != len(opens) or len(opens) != len(closes) \
      or len(closes) != len(volumes)):
    print("Length of series are not same.")
    return False
  return True
def contvert_data_to_df(symbol,timestamps, highs, lows, opens, closes, volumes):
  df = pd.DataFrame({
    "code" : s,
    "date" : pd.to_datetime(timestamps, unit="s").dt.strftime("%Y%m%d"),
    "open" : opens,
    "high" : highs,
    "low" : lows,
    "close" : closes,
    "volume" : volumes
  })
  return df

for s in symbols:
  # Call API
  data = call_yh_api(s, start_unix_date, end_unix_date)

  # Convert list to series
  timestamps, highs, lows, opens, closes, volumes = convert_data_to_series(data)

  # Valdation
  if not validate_series(timestamps, highs, lows, opens, closes, volumes):
    exit()
  df = contvert_data_to_df(s,timestamps, highs, lows, opens, closes, volumes)
  if not df["date"].isnull().any() and not df["open"].isnull().any() \
    and not df["high"].isnull().any() and not df["close"].isnull().any() \
    and not df["low"].isnull().any():
     df_all = pd.concat([df_all, df], ignore_index=True)
  else:
    print(s + "has missing data. Skipped.")
  # df for one symbol
  # Dataframe consists of many series
  # ensures the number of data in series are same
  # df = pd.DataFrame({
  #   "code" : s,
  #   "date" : pd.to_datetime(timestamps, unit="s").dt.strftime("%Y%m%d"),
  #   "open" : opens,
  #   "high" : highs,
  #   "low" : lows,
  #   "close" : closes,
  #   "volume" : volumes
  # })
  # append


df_all.to_csv("OHLC.csv", index=False)
print("OHLC.csv is generated.")

https://query1.finance.yahoo.com/v8/finance/chart/1299.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/1788.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/9988.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/2828.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/2800.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/3033.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/1810.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.finance.yahoo.com/v8/finance/chart/3690.HK?period1=1657237004&period2=1751931404&interval=1d&events=history
https://query1.f

TypeError: 'NoneType' object is not subscriptable